In [ ]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint, HuggingFaceEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnableSequence, RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv

In [ ]:
yt_api = YouTubeTranscriptApi()
video_id = 'd2kxUVwWWwU'
try:
    video_transcript = yt_api.fetch(video_id=video_id,languages=['en'])

    transcript = " ".join(text.text for text in video_transcript)
    print(transcript)
except TranscriptsDisabled:
    print("No captions available for this video.")

In [ ]:
video_transcript

In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.create_documents([transcript])
len(chunks)

In [ ]:
chunks[0]

In [ ]:
embedding = HuggingFaceEmbeddings(model='sentence-transformers/all-MiniLM-L6-v2')
vector_store = FAISS.from_documents(chunks,embedding)

In [ ]:
vector_store.index_to_docstore_id

In [ ]:
vector_store.get_by_ids(['aea8f69c-8456-47d3-bafc-07918266d72b'])

In [ ]:
retriever = vector_store.as_retriever(search_type='similarity',search_kwargs={'k':4})

In [ ]:
retriever.invoke("what is weight")

In [ ]:
llm = HuggingFaceEndpoint(
    repo_id='deepseek-ai/DeepSeek-V3.2-Exp',
    task='text-generation',
    server_kwargs={
        'temperature':0.2
    }
)

In [ ]:
model = ChatHuggingFace(llm=llm)
model

In [ ]:
prompt = PromptTemplate(
    template='''
    You are a helpful assistant.
    Answer ONLY from the provided transcript context.
    If the context is insufficient, just say you don't know.
    
    {context}
    Question: {question}
    ''',
    input_variables=['context','question']
)

In [ ]:
question = 'What is bias'
retrieved_docs = retriever.invoke(question)

In [ ]:
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)

In [ ]:
context_text

In [ ]:
final_prompt = prompt.invoke({'context':context_text,'question':question})

In [ ]:
final_prompt

In [ ]:
answer = model.invoke(final_prompt)
print(answer.content)

In [ ]:
def format_docs(retrieved_docs):
    context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
    return context_text

In [ ]:
parallel_chain = RunnableParallel({
    'context': retriever | RunnableLambda(format_docs),
    'question':RunnablePassthrough()
})

In [ ]:
parser = StrOutputParser()

In [ ]:
final_chain = parallel_chain | prompt | model | parser
result = final_chain.invoke('what is sigmoid function')
result